In [12]:
import requests
from bs4 import BeautifulSoup
import selenium
import pandas
from selenium import webdriver
from datetime import datetime, date, time
import time


In [13]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
#options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

In [14]:
url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089.htm?filter.iso3Language=eng"
driver.get(url)

In [15]:
html_soup = BeautifulSoup(driver.page_source, "html.parser")
containers = html_soup.find_all("div", class_="gdReview")

userEmail = "hassan.tariq@aptiv.com"
userPassword = "REKucuV4"

driver.find_element_by_xpath("/html/body/header/nav[1]/div/div/div/div[1]/button").click()
time.sleep(5)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[1]/div/div/input").send_keys(userEmail)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[2]/div/div/input").send_keys(userPassword)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[3]/div[1]/button").click()
time.sleep(5)
# read = driver.find_element_by_class_name("v2__EIReviewDetailsV2__continueReading v2__EIReviewDetailsV2__clickable v2__EIReviewDetailsV2__newUiCta mb")
# read.click()

def scraper(num_reviews):

    reviews = []
    for container in containers:
        rating = container.find('span', class_='ratingNumber mr-xsm')
        rating = rating.text
        print('star reviews:', rating)

        employee_Status = container.find('span', class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0')
        employee_Status = employee_Status.text
        print('employee_status:', employee_Status)
        
        #print(html_soup.select('h2.mb-xxsm')[0].get_text())
        # for i,j in zip(html_soup.select('h2.mb-xxsm'),range(len(html_soup.select('h2.mb-xxsm')))):
        #     print(html_soup.select('h2.mb-xxsm')[j].get_text())

        review_title = container.find('a', class_="reviewLink")
        review_title = review_title.text
        print('review title:', review_title)

        date_posted = container.find('span', class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey')
        date_posted = date_posted.text
        print('date & position: ', date_posted)

        author_location = container.find('span', class_='authorLocation')
        author_location = author_location.text
        print('author location:', author_location)
        
        rev_text = container.find('div', class_="v2__EIReviewDetailsV2__fullWidth")
        pros = rev_text.text
        pros = pros.strip('Pros')
        cons = rev_text.next_sibling.text
        cons = cons.strip('Cons')
        advise = rev_text.next_sibling.next_sibling.text
        advise = advise.strip('Advice to Management')
        print('pros: ', pros)
        print('cons: ', cons)
        print('advise: ', advise)

        # cont_reading = container.find('div', class_='v2__EIReviewDetailsV2__clickable')
        # cont_reading = cont_reading.click
        # print(cont_reading)

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of reviews. Needed {}, got {}.".format(num_reviews, len(reviews)))
            break

        print()

SyntaxError: invalid syntax (<ipython-input-15-44c3ea059da9>, line 16)